# Amadeus API to get Flight Offers

- [get-started-with-amadeus-apis](https://developers.amadeus.com/get-started/get-started-with-self-service-apis-335)
- [list-of-amadeus-apis](https://developers.amadeus.com/self-service)
- [flight-offers-api](https://developers.amadeus.com/self-service/category/flights/api-doc/flight-offers-search/api-reference)

In [13]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import itertools
import json
import os

In [14]:
# Fonctions pour obtenir le token d'accès et récupérer les données des vols
def get_access_token(client_id, client_secret):
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception(f"Failed to get access token: {response.status_code}, {response.text}")

def get_flight_data(access_token, origin, destination, departure_date):
    url = f"https://test.api.amadeus.com/v2/shopping/flight-offers"
    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": departure_date,
        "adults": 1,
        "nonStop": "false",
        "currencyCode": "EUR",
        "max": 100
    }
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers, params=params)
    flights = []
    if response.status_code == 200:
        data = response.json()
        for offer in data['data']:
            for itinerary in offer['itineraries']:
                for segment in itinerary['segments']:
                    flights.append({
                        'departure_city': origin,
                        'destination_city': destination,
                        'departure_date': departure_date,
                        'duration': itinerary['duration'],
                        'carrier_code': segment['carrierCode'],
                        'flight_number': segment['number'],
                        'departure_time': segment['departure']['at'],
                        'arrival_time': segment['arrival']['at'],
                        'stops': segment['numberOfStops'],
                        'price': offer['price']['total']
                    })
    else:
        print(f"Failed to fetch flight data: {response.status_code}, {response.text}")
    return flights

# Sauvegarder l'état et les données des vols
def save_state(state, filename="state.json"):
    with open(filename, 'w') as f:
        json.dump(state, f)

def load_state(filename="state.json"):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return None

def save_flight_data(flight_data, filename="flight_data.csv"):
    flight_df = pd.DataFrame(flight_data)
    flight_df.to_csv(filename, index=False)

def load_flight_data(filename="flight_data.csv"):
    try:
        flight_df = pd.read_csv(filename)
        return flight_df.to_dict('records')
    except FileNotFoundError:
        return []

In [15]:
# API Key et Secret pour Amadeus
client_id = "IGnGRFWIONOz3O5GcB97FMh9GFgOYCGG"
client_secret = "f0ao9LtaW4UZiaU8"

# Obtenir le token d'accès
access_token = get_access_token(client_id, client_secret)

# Liste des villes (codes IATA)
cities = [
    'NYC', 'LON', 'PAR', 'TYO', 'SYD', 
    'BER', 'ROM', 'MAD', 'AMS', 'SIN'
    # Ajoutez d'autres codes IATA ici pour atteindre 80 villes
]

# Génération des combinaisons de départ et d'arrivée
city_combinations = list(itertools.permutations(cities, 2))

# Charger l'état précédent s'il existe
state = load_state()
if state:
    last_origin = state['last_origin']
    last_destination = state['last_destination']
    last_date_str = state['last_date']
    flight_data = load_flight_data()
else:
    last_origin = None
    last_destination = None
    last_date_str = None
    flight_data = []

# Si une date a été sauvegardée, commencer à partir de cette date
if last_date_str:
    start_date = datetime.strptime(last_date_str, '%Y-%m-%d')
else:
    start_date = datetime(2024, 7, 1)

end_date = datetime(2024, 7, 31)
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days + 1)]

# Parcourir les combinaisons de villes et les dates pour récupérer les données
for origin, destination in city_combinations:
    if last_origin and last_destination and (origin != last_origin or destination != last_destination):
        continue

    for date in date_generated:
        if last_date_str and date.strftime('%Y-%m-%d') <= last_date_str:
            continue

        date_str = date.strftime('%Y-%m-%d')
        flight_data.extend(get_flight_data(access_token, origin, destination, date_str))

        # Sauvegarder l'état après chaque requête
        state = {
            'last_origin': origin,
            'last_destination': destination,
            'last_date': date_str,
            'flight_data': flight_data
        }
        save_state(state)
        save_flight_data(flight_data)

# Convertir les données en DataFrame
flight_df = pd.DataFrame(flight_data)

# Sauvegarder les données dans un fichier Excel
with pd.ExcelWriter('travel_data.xlsx') as writer:
    flight_df.to_excel(writer, sheet_name='Flights', index=False)

# Supprimer l'état sauvegardé après la fin du script
if os.path.exists("state.json"):
    os.remove("state.json")

print("Données sauvegardées dans 'travel_data.xlsx'")

KeyboardInterrupt: 

In [16]:
flight_data[-1]

{'departure_city': 'NYC',
 'destination_city': 'LON',
 'departure_date': '2024-07-04',
 'duration': 'PT12H20M',
 'carrier_code': 'AZ',
 'flight_number': '212',
 'departure_time': '2024-07-05T09:25:00',
 'arrival_time': '2024-07-05T11:00:00',
 'stops': 0,
 'price': '538.61'}